In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
#parâmetros
#é o número de imagens que serão alimentadas na rede por vez durante o treinamento.
batch_size = 64
#é o número de vezes que a rede irá percorrer todo o conjunto de dados durante o treinamento.
epochs = 100
#é o formato das imagens de entrada que serão usadas na GAN.
input_shape = (64, 64, 3)
#é a dimensão do espaço latente, que é a entrada para o gerador da GAN.
latent_dim = 100

In [3]:
# carregar dados
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()

# filtrar imagens de gatos e não-gatos
cat_indices = np.where(train_labels == 3)[0]
notcat_indices = np.where(train_labels != 3)[0]
cat_images = train_images[cat_indices]
notcat_images = train_images[notcat_indices]

# redimensionar imagens
cat_images = tf.image.resize(cat_images, (input_shape[0], input_shape[1]))
notcat_images = tf.image.resize(notcat_images, (input_shape[0], input_shape[1]))

# normalizar imagens, dividindo cada pixel por 127.5 e subtraindo 1, para que todos os pixels estejam na faixa de -1 a 1
cat_images = (cat_images.numpy().astype('float32') - 127.5) / 127.5
notcat_images = (notcat_images.numpy().astype('float32') - 127.5) / 127.5

# combinar imagens de gatos e não-gatos em um único conjunto de dados, 0 para gatos e 1 para não-gatos
train_images = np.concatenate([cat_images, notcat_images], axis=0)
train_labels = np.concatenate([np.zeros(len(cat_images)), np.ones(len(notcat_images))])


In [4]:
# definir gerador
generator = tf.keras.models.Sequential([
    tf.keras.layers.Dense(8 * 8 * 256, input_dim=latent_dim),
    tf.keras.layers.Reshape((8, 8, 256)),
    tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh')
])

# definir discriminador
discriminator = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [5]:
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))
generator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))
gan = tf.keras.models.Sequential([generator, discriminator])
discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

In [6]:
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [7]:
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    x = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    return x, y

In [8]:
def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    x = dataset[ix]
    y = np.ones((n_samples, 1))
    return x, y

In [24]:
def generate_and_show_images(model, epoch, latent_dim, n_examples=25, fig_size=(10,10)):
    # gerar pontos latentes
    x_input = generate_latent_points(latent_dim, n_examples)
    # gerar imagens falsas
    X = model.predict(x_input)
    # normalizar valores de pixel para o intervalo [0, 1]
    X = (X - X.min()) / (X.max() - X.min())
    # plotar imagens geradas
    plt.figure(figsize=fig_size)
    for i in range(n_examples):
        # definir subplot
        plt.subplot(5, 5, 1 + i)
        # desativar os eixos
        plt.axis('off')
        # plotar imagem
        plt.imshow(X[i, :, :, :])
    # salvar a figura
    plt.savefig('generated_plot_e%03d.png' % (epoch+1))
    # mostrar a figura
    #plt.show()
    #plt.close()

In [25]:
def train_gan(generator, discriminator, gan, dataset, latent_dim, n_epochs=100, n_batch=64):
    batch_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            x_real, y_real = generate_real_samples(dataset, half_batch)
            d_loss_real = discriminator.train_on_batch(x_real, y_real)
            x_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
            d_loss_fake = discriminator.train_on_batch(x_fake, y_fake)
            x_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))
            g_loss = gan.train_on_batch(x_gan, y_gan)
            print('>%d, %d/%d, d_real=%.3f, d_fake=%.3f g=%.3f' % (i+1, j+1, batch_per_epoch, d_loss_real, d_loss_fake, g_loss))
            #generate_and_show_images(generator, i, latent_dim)

In [26]:
train_gan(generator, discriminator, gan, train_images, latent_dim, epochs, batch_size)

1/1 [==============================] - 0s 82ms/step
>1, 1/781, d_real=0.000, d_fake=0.003 g=6.488
1/1 [==============================] - 0s 82ms/step
>1, 2/781, d_real=0.000, d_fake=0.003 g=6.981
1/1 [==============================] - 0s 82ms/step
>1, 3/781, d_real=0.003, d_fake=0.002 g=6.856
1/1 [==============================] - 0s 70ms/step
>1, 4/781, d_real=0.103, d_fake=0.015 g=5.105
1/1 [==============================] - 0s 71ms/step
>1, 5/781, d_real=0.000, d_fake=0.024 g=6.016
1/1 [==============================] - 0s 78ms/step
>1, 6/781, d_real=0.086, d_fake=0.005 g=5.784
1/1 [==============================] - 0s 80ms/step
>1, 7/781, d_real=0.001, d_fake=0.005 g=5.957
1/1 [==============================] - 0s 80ms/step
>1, 8/781, d_real=0.136, d_fake=0.014 g=4.894
1/1 [==============================] - 0s 76ms/step
>1, 9/781, d_real=0.000, d_fake=0.011 g=5.416
1/1 [==============================] - 0s 72ms/step
>1, 10/781, d_real=0.008, d_fake=0.007 g=5.822
1/1 [==============

1/1 [==============================] - 0s 86ms/step
>1, 84/781, d_real=0.000, d_fake=0.000 g=25.484
1/1 [==============================] - 0s 72ms/step
>1, 85/781, d_real=0.698, d_fake=0.000 g=17.951
1/1 [==============================] - 0s 71ms/step
>1, 86/781, d_real=0.775, d_fake=0.000 g=11.852
1/1 [==============================] - 0s 71ms/step
>1, 87/781, d_real=0.000, d_fake=0.000 g=8.957
1/1 [==============================] - 0s 82ms/step
>1, 88/781, d_real=0.011, d_fake=0.003 g=6.049
1/1 [==============================] - 0s 74ms/step
>1, 89/781, d_real=0.000, d_fake=2.133 g=18.208
1/1 [==============================] - 0s 80ms/step
>1, 90/781, d_real=0.000, d_fake=0.000 g=28.312
1/1 [==============================] - 0s 77ms/step
>1, 91/781, d_real=0.636, d_fake=0.000 g=23.531
1/1 [==============================] - 0s 85ms/step
>1, 92/781, d_real=0.653, d_fake=0.000 g=17.504
1/1 [==============================] - 0s 85ms/step
>1, 93/781, d_real=0.263, d_fake=0.000 g=12.433
1/

1/1 [==============================] - 0s 83ms/step
>1, 166/781, d_real=0.127, d_fake=0.146 g=3.206
1/1 [==============================] - 0s 87ms/step
>1, 167/781, d_real=0.173, d_fake=0.116 g=3.158
1/1 [==============================] - 0s 99ms/step
>1, 168/781, d_real=0.111, d_fake=0.099 g=3.161
1/1 [==============================] - 0s 126ms/step
>1, 169/781, d_real=0.053, d_fake=0.101 g=3.284
1/1 [==============================] - 0s 83ms/step
>1, 170/781, d_real=0.164, d_fake=0.111 g=3.852
1/1 [==============================] - 0s 81ms/step
>1, 171/781, d_real=0.131, d_fake=0.074 g=3.355
1/1 [==============================] - 0s 93ms/step
>1, 172/781, d_real=0.088, d_fake=0.110 g=3.637
1/1 [==============================] - 0s 87ms/step
>1, 173/781, d_real=0.118, d_fake=0.075 g=3.794
1/1 [==============================] - 0s 96ms/step
>1, 174/781, d_real=0.071, d_fake=0.091 g=4.142
1/1 [==============================] - 0s 95ms/step
>1, 175/781, d_real=0.008, d_fake=0.036 g=4.675

1/1 [==============================] - 0s 103ms/step
>1, 248/781, d_real=0.352, d_fake=0.294 g=1.805
1/1 [==============================] - 0s 87ms/step
>1, 249/781, d_real=0.291, d_fake=0.337 g=2.214
1/1 [==============================] - 0s 88ms/step
>1, 250/781, d_real=0.427, d_fake=0.275 g=2.293
1/1 [==============================] - 0s 78ms/step
>1, 251/781, d_real=0.427, d_fake=0.190 g=2.064
1/1 [==============================] - 0s 86ms/step
>1, 252/781, d_real=0.365, d_fake=0.362 g=1.912
1/1 [==============================] - 0s 84ms/step
>1, 253/781, d_real=0.388, d_fake=0.372 g=2.245
1/1 [==============================] - 0s 90ms/step
>1, 254/781, d_real=0.436, d_fake=0.210 g=2.233
1/1 [==============================] - 0s 89ms/step
>1, 255/781, d_real=0.354, d_fake=0.298 g=2.110
1/1 [==============================] - 0s 85ms/step
>1, 256/781, d_real=0.420, d_fake=0.335 g=2.081
1/1 [==============================] - 0s 103ms/step
>1, 257/781, d_real=0.226, d_fake=0.293 g=2.29

1/1 [==============================] - 0s 75ms/step
>1, 330/781, d_real=0.621, d_fake=0.605 g=1.474
1/1 [==============================] - 0s 83ms/step
>1, 331/781, d_real=0.559, d_fake=0.367 g=1.491
1/1 [==============================] - 0s 83ms/step
>1, 332/781, d_real=0.742, d_fake=0.699 g=1.341
1/1 [==============================] - 0s 110ms/step
>1, 333/781, d_real=0.372, d_fake=0.443 g=1.557
1/1 [==============================] - 0s 85ms/step
>1, 334/781, d_real=0.535, d_fake=0.438 g=1.530
1/1 [==============================] - 0s 170ms/step
>1, 335/781, d_real=0.591, d_fake=0.526 g=1.302
1/1 [==============================] - 0s 105ms/step
>1, 336/781, d_real=0.511, d_fake=0.588 g=1.448
1/1 [==============================] - 0s 160ms/step
>1, 337/781, d_real=0.418, d_fake=0.463 g=1.542
1/1 [==============================] - 0s 132ms/step
>1, 338/781, d_real=0.630, d_fake=0.518 g=1.509
1/1 [==============================] - 0s 77ms/step
>1, 339/781, d_real=0.636, d_fake=0.451 g=1

...

1/1 [==============================] - 0s 81ms/step
>6, 447/781, d_real=0.721, d_fake=0.573 g=1.088
1/1 [==============================] - 0s 81ms/step
>6, 448/781, d_real=0.663, d_fake=0.688 g=0.942
1/1 [==============================] - 0s 80ms/step
>6, 449/781, d_real=0.598, d_fake=0.598 g=0.976
1/1 [==============================] - 0s 84ms/step
>6, 450/781, d_real=0.685, d_fake=0.799 g=0.972
1/1 [==============================] - 0s 80ms/step
>6, 451/781, d_real=0.635, d_fake=0.666 g=1.040
1/1 [==============================] - 0s 84ms/step
>6, 452/781, d_real=0.652, d_fake=0.590 g=0.948
1/1 [==============================] - 0s 85ms/step
>6, 453/781, d_real=0.585, d_fake=0.591 g=1.031
1/1 [==============================] - 0s 81ms/step
>6, 454/781, d_real=0.628, d_fake=0.584 g=1.003
1/1 [==============================] - 0s 84ms/step
>6, 455/781, d_real=0.656, d_fake=0.698 g=0.996
1/1 [==============================] - 0s 80ms/step
>6, 456/781, d_real=0.630, d_fake=0.695 g=1.061


1/1 [==============================] - 0s 80ms/step
>6, 529/781, d_real=0.705, d_fake=0.571 g=1.025
1/1 [==============================] - 0s 83ms/step
>6, 530/781, d_real=0.620, d_fake=0.646 g=0.977
1/1 [==============================] - 0s 82ms/step
>6, 531/781, d_real=0.722, d_fake=0.676 g=0.973
1/1 [==============================] - 0s 81ms/step
>6, 532/781, d_real=0.636, d_fake=0.687 g=0.975
1/1 [==============================] - 0s 81ms/step
>6, 533/781, d_real=0.573, d_fake=0.674 g=1.008
1/1 [==============================] - 0s 81ms/step
>6, 534/781, d_real=0.672, d_fake=0.584 g=1.040
1/1 [==============================] - 0s 84ms/step
>6, 535/781, d_real=0.694, d_fake=0.662 g=0.979
1/1 [==============================] - 0s 79ms/step
>6, 536/781, d_real=0.659, d_fake=0.676 g=0.970
1/1 [==============================] - 0s 82ms/step
>6, 537/781, d_real=0.752, d_fake=0.654 g=0.983
1/1 [==============================] - 0s 79ms/step
>6, 538/781, d_real=0.747, d_fake=0.583 g=0.969


1/1 [==============================] - 0s 83ms/step
>6, 611/781, d_real=0.564, d_fake=0.611 g=0.999
1/1 [==============================] - 0s 85ms/step
>6, 612/781, d_real=0.582, d_fake=0.516 g=1.014
1/1 [==============================] - 0s 88ms/step
>6, 613/781, d_real=0.570, d_fake=0.789 g=1.066
1/1 [==============================] - 0s 85ms/step
>6, 614/781, d_real=0.740, d_fake=0.554 g=1.070
1/1 [==============================] - 0s 83ms/step
>6, 615/781, d_real=0.575, d_fake=0.654 g=1.054
1/1 [==============================] - 0s 81ms/step
>6, 616/781, d_real=0.763, d_fake=0.599 g=0.967
1/1 [==============================] - 0s 84ms/step
>6, 617/781, d_real=0.559, d_fake=0.693 g=0.992
1/1 [==============================] - 0s 84ms/step
>6, 618/781, d_real=0.615, d_fake=0.833 g=1.049
1/1 [==============================] - 0s 85ms/step
>6, 619/781, d_real=0.676, d_fake=0.541 g=1.165
1/1 [==============================] - 0s 85ms/step
>6, 620/781, d_real=0.639, d_fake=0.470 g=1.131


1/1 [==============================] - 0s 82ms/step
>6, 693/781, d_real=0.577, d_fake=0.477 g=1.095
1/1 [==============================] - 0s 88ms/step
>6, 694/781, d_real=0.575, d_fake=0.632 g=1.133
1/1 [==============================] - 0s 85ms/step
>6, 695/781, d_real=0.630, d_fake=0.487 g=1.212
1/1 [==============================] - 0s 85ms/step
>6, 696/781, d_real=0.560, d_fake=0.547 g=1.202
1/1 [==============================] - 0s 86ms/step
>6, 697/781, d_real=0.788, d_fake=0.549 g=1.028
1/1 [==============================] - 0s 93ms/step
>6, 698/781, d_real=0.614, d_fake=0.702 g=1.092
1/1 [==============================] - 0s 88ms/step
>6, 699/781, d_real=0.583, d_fake=0.496 g=1.091
1/1 [==============================] - 0s 120ms/step
>6, 700/781, d_real=0.590, d_fake=0.556 g=1.118
1/1 [==============================] - 0s 95ms/step
>6, 701/781, d_real=0.554, d_fake=0.536 g=1.109
1/1 [==============================] - 0s 95ms/step
>6, 702/781, d_real=0.590, d_fake=0.566 g=1.104

KeyboardInterrupt: 